In [36]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  , ConfusionMatrixDisplay , classification_report , f1_score , precision_score , recall_score , confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import seaborn as sns
import time

from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model
import keras

In [2]:
categories = ['jogging' , 
              'sitting' , 
              'standing' , 
              'walking']

data_directory = '/kaggle/input/data1/Dataset_images'


In [3]:
# Step 2: Data Preprocessing
image_size = (256, 256)
dataset = []
labels = []

for category in categories:
    folder_path = os.path.join(data_directory, category)
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = load_img(image_path, target_size=image_size)
        image_array = img_to_array(image)
        dataset.append(image_array)
        labels.append(categories.index(category))

dataset = np.array(dataset)
labels = np.array(labels)

In [5]:
train_data, val_data, train_labels, val_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42)

In [6]:
# Step 4: CNN Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])

In [7]:
# Step 5: Model Compilation
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10, batch_size=32)

Epoch 1/10
65/65 [==============================] - 14s 75ms/step - loss: 58.6254 - accuracy: 0.5873 - val_loss: 0.5287 - val_accuracy: 0.8066
Epoch 2/10
65/65 [==============================] - 4s 55ms/step - loss: 0.3981 - accuracy: 0.8549 - val_loss: 0.5084 - val_accuracy: 0.8317
Epoch 3/10
65/65 [==============================] - 4s 60ms/step - loss: 0.1637 - accuracy: 0.9400 - val_loss: 0.3781 - val_accuracy: 0.8975
Epoch 4/10
65/65 [==============================] - 4s 58ms/step - loss: 0.0706 - accuracy: 0.9734 - val_loss: 0.3997 - val_accuracy: 0.9014
Epoch 5/10
65/65 [==============================] - 4s 55ms/step - loss: 0.0321 - accuracy: 0.9889 - val_loss: 0.5455 - val_accuracy: 0.8820
Epoch 6/10
65/65 [==============================] - 4s 54ms/step - loss: 0.0783 - accuracy: 0.9763 - val_loss: 0.6953 - val_accuracy: 0.9110
Epoch 7/10
65/65 [==============================] - 3s 53ms/step - loss: 0.4878 - accuracy: 0.8162 - val_loss: 1.2362 - val_accuracy: 0.6441
Epoch 8/10


In [13]:
folder_path = '/kaggle/input/test-images/test_images'


# Get the list of image file names in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Initialize an empty list to store the resized images
test_images = []

# Specify the desired image size
desired_size = (256, 256)  

# Loop through the image files
for image_file in image_files:
    # Read the image using OpenCV
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    # Resize the image to the desired size
    resized_image = cv2.resize(image, desired_size)

    # Append the resized image to the list
    test_images.append(resized_image)

In [14]:
# Convert the list of images to a NumPy array
test_images = np.array(test_images)
# Print the shape of the image array
print("Image array shape:", test_images.shape)

Image array shape: (100, 256, 256, 3)


In [32]:

# Set up a timer
start_time = time.time()

# Number of frames processed
frame_count = 0

# Inference loop
for frame in test_images:
    
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)

    frame_count += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate FPS
fps = frame_count / elapsed_time

# Print the FPS
print("FPS: {:.2f}".format(fps))

4/4 [==============================] - 0s 11ms/step
FPS: 8.55


In [33]:
from keras.applications.inception_v3 import InceptionV3
feature_extractor = InceptionV3(weights='imagenet', input_shape=(256, 256, 3), include_top=False)

87910968/87910968 [==============================] - 1s 0us/step


In [34]:
for layer in feature_extractor.layers[: -23]:
    layer.trainable = False

In [37]:
inp_layer = Input(shape = (256, 256, 3))

features = feature_extractor(inp_layer)

flat = Flatten()(features)
FC1 = Dense(units = 200, activation = 'relu')(flat)
FC2 = Dense(units = 100, activation = 'relu')(FC1)
output = Dense(units = 4, activation = 'softmax')(FC2)

model = Model(inputs = inp_layer, outputs = output)


In [38]:
# Step 5: Model Compilation
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10, batch_size=32)

Epoch 1/10
65/65 [==============================] - 19s 155ms/step - loss: 3.3187 - accuracy: 0.7770 - val_loss: 6.1070 - val_accuracy: 0.7041
Epoch 2/10
65/65 [==============================] - 5s 80ms/step - loss: 0.1978 - accuracy: 0.9473 - val_loss: 1.3508 - val_accuracy: 0.8414
Epoch 3/10
65/65 [==============================] - 5s 81ms/step - loss: 0.0707 - accuracy: 0.9777 - val_loss: 0.6066 - val_accuracy: 0.8897
Epoch 4/10
65/65 [==============================] - 6s 85ms/step - loss: 0.1812 - accuracy: 0.9555 - val_loss: 0.2072 - val_accuracy: 0.9536
Epoch 5/10
65/65 [==============================] - 6s 86ms/step - loss: 0.1410 - accuracy: 0.9647 - val_loss: 0.3768 - val_accuracy: 0.9381
Epoch 6/10
65/65 [==============================] - 5s 80ms/step - loss: 0.0630 - accuracy: 0.9792 - val_loss: 0.3896 - val_accuracy: 0.9458
Epoch 7/10
65/65 [==============================] - 5s 83ms/step - loss: 0.1442 - accuracy: 0.9729 - val_loss: 0.7649 - val_accuracy: 0.8839
Epoch 8/10


In [39]:
# Set up a timer
start_time = time.time()

# Number of frames processed
frame_count = 0

# Inference loop
for frame in test_images:
    
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)

    frame_count += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate FPS
fps = frame_count / elapsed_time

# Print the FPS
print("FPS: {:.2f}".format(fps))


4/4 [==============================] - 0s 48ms/step
FPS: 3.11
